# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f99989f7df0>
├── 'a' --> tensor([[-0.5859,  0.9555,  0.8731],
│                   [ 0.5888,  1.0739,  0.2829]])
└── 'x' --> <FastTreeValue 0x7f99989f7f70>
    └── 'c' --> tensor([[ 1.6059, -0.3826,  0.8560,  0.4802],
                        [-1.5158, -1.1513,  0.6918,  0.9642],
                        [ 0.2429,  1.0502, -0.2264,  1.8137]])

In [4]:
t.a

tensor([[-0.5859,  0.9555,  0.8731],
        [ 0.5888,  1.0739,  0.2829]])

In [5]:
%timeit t.a

58.2 ns ± 0.044 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f99989f7df0>
├── 'a' --> tensor([[-0.2637,  0.1461, -0.0046],
│                   [-1.1276,  1.5740, -1.2817]])
└── 'x' --> <FastTreeValue 0x7f99989f7f70>
    └── 'c' --> tensor([[ 1.6059, -0.3826,  0.8560,  0.4802],
                        [-1.5158, -1.1513,  0.6918,  0.9642],
                        [ 0.2429,  1.0502, -0.2264,  1.8137]])

In [7]:
%timeit t.a = new_value

60.2 ns ± 0.0335 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.5859,  0.9555,  0.8731],
               [ 0.5888,  1.0739,  0.2829]]),
    x: Batch(
           c: tensor([[ 1.6059, -0.3826,  0.8560,  0.4802],
                      [-1.5158, -1.1513,  0.6918,  0.9642],
                      [ 0.2429,  1.0502, -0.2264,  1.8137]]),
       ),
)

In [10]:
b.a

tensor([[-0.5859,  0.9555,  0.8731],
        [ 0.5888,  1.0739,  0.2829]])

In [11]:
%timeit b.a

57.2 ns ± 0.012 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.2188,  0.7219,  1.5771],
               [ 0.9133,  0.4214, -0.1858]]),
    x: Batch(
           c: tensor([[ 1.6059, -0.3826,  0.8560,  0.4802],
                      [-1.5158, -1.1513,  0.6918,  0.9642],
                      [ 0.2429,  1.0502, -0.2264,  1.8137]]),
       ),
)

In [13]:
%timeit b.a = new_value

490 ns ± 0.119 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

827 ns ± 0.0663 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 25.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 236 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 837 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f9962bcf280>
├── 'a' --> tensor([[[-0.5859,  0.9555,  0.8731],
│                    [ 0.5888,  1.0739,  0.2829]],
│           
│                   [[-0.5859,  0.9555,  0.8731],
│                    [ 0.5888,  1.0739,  0.2829]],
│           
│                   [[-0.5859,  0.9555,  0.8731],
│                    [ 0.5888,  1.0739,  0.2829]],
│           
│                   [[-0.5859,  0.9555,  0.8731],
│                    [ 0.5888,  1.0739,  0.2829]],
│           
│                   [[-0.5859,  0.9555,  0.8731],
│                    [ 0.5888,  1.0739,  0.2829]],
│           
│                   [[-0.5859,  0.9555,  0.8731],
│                    [ 0.5888,  1.0739,  0.2829]],
│           
│                   [[-0.5859,  0.9555,  0.8731],
│                    [ 0.5888,  1.0739,  0.2829]],
│           
│                   [[-0.5859,  0.9555,  0.8731],
│                    [ 0.5888,  1.0739,  0.2829]]])
└── 'x' --> <FastTreeValue 0x7f98eec75be0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.9 µs ± 53.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f98ee697f10>
├── 'a' --> tensor([[-0.5859,  0.9555,  0.8731],
│                   [ 0.5888,  1.0739,  0.2829],
│                   [-0.5859,  0.9555,  0.8731],
│                   [ 0.5888,  1.0739,  0.2829],
│                   [-0.5859,  0.9555,  0.8731],
│                   [ 0.5888,  1.0739,  0.2829],
│                   [-0.5859,  0.9555,  0.8731],
│                   [ 0.5888,  1.0739,  0.2829],
│                   [-0.5859,  0.9555,  0.8731],
│                   [ 0.5888,  1.0739,  0.2829],
│                   [-0.5859,  0.9555,  0.8731],
│                   [ 0.5888,  1.0739,  0.2829],
│                   [-0.5859,  0.9555,  0.8731],
│                   [ 0.5888,  1.0739,  0.2829],
│                   [-0.5859,  0.9555,  0.8731],
│                   [ 0.5888,  1.0739,  0.2829]])
└── 'x' --> <FastTreeValue 0x7f9962bcf910>
    └── 'c' --> tensor([[ 1.6059, -0.3826,  0.8560,  0.4802],
                        [-1.5158, -1.1513,  0.6918,  0.9642],
                 

In [23]:
%timeit t_cat(trees)

30.2 µs ± 52.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.3 µs ± 109 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.6059, -0.3826,  0.8560,  0.4802],
                       [-1.5158, -1.1513,  0.6918,  0.9642],
                       [ 0.2429,  1.0502, -0.2264,  1.8137]],
              
                      [[ 1.6059, -0.3826,  0.8560,  0.4802],
                       [-1.5158, -1.1513,  0.6918,  0.9642],
                       [ 0.2429,  1.0502, -0.2264,  1.8137]],
              
                      [[ 1.6059, -0.3826,  0.8560,  0.4802],
                       [-1.5158, -1.1513,  0.6918,  0.9642],
                       [ 0.2429,  1.0502, -0.2264,  1.8137]],
              
                      [[ 1.6059, -0.3826,  0.8560,  0.4802],
                       [-1.5158, -1.1513,  0.6918,  0.9642],
                       [ 0.2429,  1.0502, -0.2264,  1.8137]],
              
                      [[ 1.6059, -0.3826,  0.8560,  0.4802],
                       [-1.5158, -1.1513,  0.6918,  0.9642],
                       [ 0.2429,  1.0502, -0.2264,  1.8137]],

In [26]:
%timeit Batch.stack(batches)

78.9 µs ± 64.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.6059, -0.3826,  0.8560,  0.4802],
                      [-1.5158, -1.1513,  0.6918,  0.9642],
                      [ 0.2429,  1.0502, -0.2264,  1.8137],
                      [ 1.6059, -0.3826,  0.8560,  0.4802],
                      [-1.5158, -1.1513,  0.6918,  0.9642],
                      [ 0.2429,  1.0502, -0.2264,  1.8137],
                      [ 1.6059, -0.3826,  0.8560,  0.4802],
                      [-1.5158, -1.1513,  0.6918,  0.9642],
                      [ 0.2429,  1.0502, -0.2264,  1.8137],
                      [ 1.6059, -0.3826,  0.8560,  0.4802],
                      [-1.5158, -1.1513,  0.6918,  0.9642],
                      [ 0.2429,  1.0502, -0.2264,  1.8137],
                      [ 1.6059, -0.3826,  0.8560,  0.4802],
                      [-1.5158, -1.1513,  0.6918,  0.9642],
                      [ 0.2429,  1.0502, -0.2264,  1.8137],
                      [ 1.6059, -0.3826,  0.8560,  0.4802],
                   

In [28]:
%timeit Batch.cat(batches)

145 µs ± 146 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

303 µs ± 548 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
